<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/memoria_preprocessing_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento

El objetivo de esta sección es el de ver que técnicas existen para tratar de reducir el vocabulario de las oraciones. Con esto conseguimos acelerar y mejorar el entrenamiento de los modelos, ya que en el input de lo que vamos a codificar ya va la información esencial y la que contiene mayor carga de significado.

Podemos encontrarnos palabras con errores ortográficos, palabras que no aportan significado, palabras con terminaciones diferentes pero el mismo valor...

Si conseguimos tratar todos estos puntos vamos a conseguir extraer las características que realmente importan de los textos.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import time

import numpy as np
from numpy.random import seed
seed(9)

import tensorflow as tf
tf.random.set_seed(9)

import json
import nltk
from nltk.tokenize import WordPunctTokenizer
import keras
import re

import matplotlib.pyplot as plt
plt.style.use('default')

nltk.download("popular")
print(tf.__version__)

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [10]:
dataover = pd.read_csv("gdrive/My Drive/TFG/tmdb_spanish_def.csv", sep='#',encoding='utf-8', lineterminator='\n')
taste = pd.read_csv("gdrive/My Drive/TFG/tmdb_spanish_Jaime_def.csv", sep='#', encoding='utf-8')

In [11]:
taste = taste[~taste['id'].str.contains('/')]
taste['id'] = taste['id'].astype(int)

data = taste.merge(dataover[['id','title','overview','genres','crew','cast']], left_on='id', right_on='id')
data = data[~pd.isna(data.overview)]

data = data.dropna(subset=['like'])
data['like'] = data['like'].astype(int)
data.reset_index(inplace=True,drop=True)
print(len(data))

print(data.like.value_counts(dropna=False))

1109
0    566
1    543
Name: like, dtype: int64


## Correción ortográfica

Uno de los problema al que nos enfrentamos cuando hablamos de textos es al de los errores ortográficos, muchas veces escribimos rápido y nos pasamos por alto acentos, confundimos letras por tener sonidos similares, intercalamos letras de más, tenemos letras en mayúsculas y minúsculas, juntamos palabras...

Algunos de ellos como las tildes tienen una solución sencilla que además suelen ser los errores más habituales. Nuestra base de datos de partida es editable por los usuarios y es un punto necesario para ser tratado.

Vamos a ver las funciones y técnicas que nos van a ayudar a resolver este apartado:

### Normalización

Vamos a quitar todas las tildes y llevar todas las letras a minúsculas, de tal manera que no pueda haber diferencias entre la palabra con tílde o sin ella o si tiene una letra en mayus. o en minus.
Además vamos a eliminar todos los signos de puntuación.

In [14]:
example = data.overview[0]
example

'La princesa Leia, líder del movimiento rebelde que desea reinstaurar la República en la galaxia en los tiempos ominosos del Imperio, es capturada por las malévolas Fuerzas Imperiales, capitaneadas por el implacable Darth Vader, el sirviente más fiel del emperador. El intrépido Luke Skywalker, ayudado por Han Solo, capitán de la nave espacial "El Halcón Milenario", y los androides, R2D2 y C3PO, serán los encargados de luchar contra el enemigo y rescatar a la princesa para volver a instaurar la justicia en el seno de la Galaxia.'

In [17]:
import string

def normalize(x):
  x = x.lower()
  replacements = (
      ("á", "a"),
      ("é", "e"),
      ("í", "i"),
      ("ó", "o"),
      ("ú", "u"),
      ("ñ", "n")
  )
  for a, b in replacements:
      x = x.replace(a, b)

  x = x.translate(str.maketrans('','',string.punctuation))
  x = x.translate(str.maketrans('','','ªº¡¿'))    
  return x

In [18]:
normalize(example)

'la princesa leia lider del movimiento rebelde que desea reinstaurar la republica en la galaxia en los tiempos ominosos del imperio es capturada por las malevolas fuerzas imperiales capitaneadas por el implacable darth vader el sirviente mas fiel del emperador el intrepido luke skywalker ayudado por han solo capitan de la nave espacial el halcon milenario y los androides r2d2 y c3po seran los encargados de luchar contra el enemigo y rescatar a la princesa para volver a instaurar la justicia en el seno de la galaxia'